In [0]:
# Where are you running the notebook
# if colab == False assuming you have a local TF installation with GPU
colab = True

# How are you installing Flamedisx
# if fd_clone == True will git clone repo, otherwise use pip install
fd_clone = True

In [0]:
if colab:
    # Mount Google Drive to use XENON data w flamedisx
    from google.colab import drive
    drive.mount('/content/drive/')

    data_path = 'drive/My Drive/flamedisx_data/'

    # Install TF2 and TFP w GPU support (change runtime to GPU to test)
    !pip install -U tensorflow-gpu==2.0.0
    !pip install -U tensorflow_probability==0.8.0
else:
    data_path = ''

In [0]:
if fd_clone:
    !git clone https://github.com/JelleAalbers/flamedisx.git

    %cd flamedisx
    !git checkout master
    !git pull origin master
    !python setup.py develop
    %cd ..
else:
    !pip install flamedisx

In [0]:
import tensorflow as tf
tf.__version__, tf.test.is_built_with_cuda(), tf.test.is_gpu_available()

In [0]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from multihist import Hist1d, Histdd

In [0]:
import flamedisx as fd
import tensorflow as tf
fd.float_type()

In [0]:
rn_data = pd.read_csv(data_path + 'data_Rn220_lowenergy.csv')[::4]

rn_data = rn_data[(rn_data['s1'] < 70) & (rn_data['s1'] > 2)].copy()
rn_data = rn_data[(rn_data['s2'] > 200) & (rn_data['s2'] < 6000)].copy()

rn_data['theta'] = np.arctan2(rn_data['y'], rn_data['x'])
rn_data['r'] = (rn_data['x']**2 + rn_data['y']**2)**0.5

rn_data = fd.x1t_sr0.SR0ERSource().annotate_data(rn_data)

rn_data_full = rn_data.copy()
rn_data = rn_data[['s1', 's2'] + ['x_observed', 'y_observed', 'x', 'y', 'z', 'drift_time', 'r', 'theta']]

In [0]:
len(rn_data)

In [0]:
ll = fd.LogLikelihood(
    sources=dict(er=fd.x1t_sr0.SR0ERSource),
    batch_size=40 if tf.test.is_gpu_available() else 10,
    free_rates='er',
    data=rn_data,
    elife=(300e3, 500e3, 3),
    er_pel_a=(5., 25., 2),
    er_pel_b=(-29., -26., 2),
    er_pel_c=(31.9, 32.8, 2),
    g2=(25., 35., 2),
    n_trials=int(2e5))
len(rn_data), len(rn_data) / ll.mu_itps['er']().numpy()

In [0]:
for pname in ll.param_names:
    if pname.endswith('_rate_multiplier'):
        continue
    start, stop, n = ll.param_specs[pname]
    x = np.linspace(start, stop, 100, dtype=np.float32)
    y = np.array([ll.mu_itps['er'](**{pname: tf.constant(q)}) for q in x])
    plt.plot(x, y)
    plt.title(pname)
    plt.show()

In [0]:
guess = ll.guess()
guess['er_rate_multiplier'] = tf.constant(0.02 * 4, dtype=fd.float_type())

In [0]:
# This ensures all graphs are traced and the benchmark below is accurate
bestfit = ll.bestfit(guess)

In [0]:
%%time
bestfit = ll.bestfit(guess)

In [0]:
%%time
inv_hess = ll.inverse_hessian(bestfit)

In [0]:
ll.summary(bestfit, inverse_hessian=inv_hess)

In [0]:
n_trials = int(1e5)

sim_guess = fd.x1t_sr0.SR0ERSource(rn_data).simulate(
    n_trials,
    **{k: v for k,v in guess.items() if not k.endswith('rate_multiplier')})

sim_bf = fd.x1t_sr0.SR0ERSource(rn_data).simulate(
    n_trials, 
    **{k: v for k,v in bestfit.items() if not k.endswith('rate_multiplier')})

bins = np.linspace(0, 11, 10)
for d_sim in sim_guess, sim_bf:
    Histdd(d_sim, dimensions=(
        ('e_vis', np.linspace(0, 11, 50)),
        ('electron_produced_mle', np.linspace(0, 350, 50)),
    )).plot()
    d = rn_data_full
    plt.scatter(d['e_vis'], d['electron_produced_mle'], c='white', s=5)
    plt.show()
